In [34]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
import pandas as pd
import numpy as np

# own Modules 
from models.models import LstmMseDropout
from utils.data_loader import DataPreperatorPrediction, DataSet
from loss_module import LossModuleMse, LossModuleMle
from utils.predictor import Predictor

In [35]:
param = {
    "data" : {
        "stake_training_data" : 0.75, 
        "path" : '../../data/artifical_signals/NewBlade_with_sine.csv', 
    },
    "model" : {
        "path" : "../../models/MSE_model/test.pt",
        "input_size" : 8,
        "n_hidden_lstm" : 150,
        "sequence_size" : 50,
        "batch_size" : 8,
        "lstm_layer" : 3,
        "n_hidden_fc": 50,
        "dropout_rate": 0.2
    },
    "results": {
        "path_prediction" : "../visualisation/files/prediction/tessssst.csv",
        "path_residual" : "../visualisation/files/residuals/tesssst.csv"
    }
}

## Standarize Data
First we have to apply normalisation to data. That is because the model works on the representation given by its input vectors. The scale of those numbers is part of the representation.
We should apply the exact same scaling as for training data. That means storing the scale and offset used with your training data, and using that again. <br>
__The mean and variance for each feature of the training data with which the model was trained (stake: 0.75):__

### Mean and Variance from NewBlade Training Data

In [36]:
mean_training_data = [-5.37536613e-02, -2.53111489e-04, -8.82854465e+05, 7.79034183e+02, 1.45531178e+04, 1.37766733e+03, 6.50149764e-01] 
var_training_data = [1.25303578e-01, 1.16898690e-03, 2.86060835e+06, 1.64515717e+06, 6.85728371e+06, 3.63196175e+05, 8.21463343e-03]

### Mean and Variance from Artifical Training Data

In [37]:
mean_training_data= [-5.31764899e-02, -3.98576146e-04, -8.82773455e+05,  8.25672897e+02, 1.47034247e+04,  1.42685595e+03,  6.62155736e-01,  1.23172374e-02]
var_training_data = [1.28792583e-01, 1.21258617e-03, 2.90245238e+06, 1.72279458e+06, 6.83095901e+06, 3.12357562e+05, 3.89033076e-03, 5.01164766e+01]

## Create DataLoader

In [38]:
data_preperator = DataPreperatorPrediction(param['data']['path'], mean_training_data, var_training_data, param["model"]["input_size"])
preprocessed_data = data_preperator.provide_data()
dataset = DataSet(preprocessed_data, timesteps=param["model"]["sequence_size"])
data_loader = DataLoader(dataset, batch_size=param["model"]["batch_size"], num_workers=1, shuffle=False, drop_last=True)

## Define Model and load Parameters of trained model

In [39]:
model = LstmMseDropout(batch_size=param['model']['batch_size'], 
                       input_dim=param['model']['input_size'], 
                       n_hidden_lstm=param['model']['n_hidden_lstm'], 
                       n_hidden_fc=param['model']['n_hidden_fc'], 
                       n_layers=param['model']['lstm_layer'], 
                       dropout_rate= param['model']['dropout_rate'])

checkpoint = torch.load(param["model"]["path"])
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

LstmMseDropout(
  (lstm): LSTM(8, 150, num_layers=3, batch_first=True, dropout=0.2)
  (fc1): Linear(in_features=150, out_features=50, bias=True)
  (dropout): Dropout(p=0.2)
  (fc2): Linear(in_features=50, out_features=8, bias=True)
)

## Define Loss Function

In [40]:
criterion = LossModuleMse(param["model"]["input_size"], param["model"]["batch_size"])

## Initialize Trainer

In [41]:
predictor = Predictor(model=model,
                      criterion=criterion,
                      path_data=param["data"]["path"],
                      path_results=param["results"]["path_prediction"],
                      path_residuals=param["results"]["path_residual"],
                      )

## Predict

In [42]:
results = predictor.predict(data_loader)
predictor.save_prediction(results)

Start predicting


Traceback (most recent call last):
  File "/Users/carlovoss/anaconda3/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/Users/carlovoss/anaconda3/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/Users/carlovoss/anaconda3/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/Users/carlovoss/anaconda3/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe


KeyboardInterrupt: 

## Compute Residuals for New Blade

In [10]:
residuals = predictor.compute_residuals(results)
predictor.save_residuals(residuals)